In [2]:
import pandas as pd
import os
import numpy as np
import librosa
from sklearn.preprocessing import OneHotEncoder

In [3]:
paths=[]
labels=[]
for filename in os.listdir('./AudioWAV'):
    
    paths.append('./AudioWAV/' + filename)
    file = filename.split('.')[0]
   
    label = file.split('_')[2]
    if label == 'ANG':
        labels.append('angry.wav')
    elif label == 'DIS':
        labels.append('disgust.wav')
    elif label == 'FEA':
        labels.append('fear.wav')
    elif label == 'HAP':
        labels.append('happy.wav')
    elif label == 'NEU':
        labels.append('neutral.wav')
    elif label == 'SAD':
        labels.append('sad.wav')
        

df_cremad = pd.DataFrame({'speech':paths,'label':labels})
df_cremad.sample(5)

,speech,label
957,./AudioWAV/1012_TIE_NEU_XX.wav,neutral.wav
2947,./AudioWAV/1037_IEO_FEA_MD.wav,fear.wav
1459,./AudioWAV/1018_WSI_FEA_XX.wav,fear.wav
5577,./AudioWAV/1069_IEO_SAD_LO.wav,sad.wav
4358,./AudioWAV/1054_ITH_HAP_XX.wav,happy.wav


In [4]:
def MFCC(filename):
    y, sr = librosa.load(filename,duration=3,offset=0.5)
    return np.mean(librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40).T,axis=0)

mfcc_cremad = df_cremad['speech'].apply(lambda x:MFCC(x))

In [5]:
X =[x for x in mfcc_cremad]
X =np.array(X)
X.shape
X =np.expand_dims(X,-1)
X.shape

(7442, 40, 1)

In [6]:
ohe=OneHotEncoder()
y = ohe.fit_transform(df_cremad[['label']] )
y = y.toarray()

In [7]:
X.shape, y.shape

((7442, 40, 1), (7442, 6))

In [8]:
df_cremad['label'].unique()

array(['angry.wav', 'disgust.wav', 'fear.wav', 'happy.wav', 'neutral.wav',
       'sad.wav'], dtype=object)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Definindo os modelos

class SimpleDNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleDNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

class LSTMAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.attention = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        attn_weights = torch.softmax(self.attention(out), dim=1)
        out = torch.sum(attn_weights * out, dim=1)
        out = self.fc(out)
        return out

class CNNModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.fc_input_size = 32 * 1 * 1
        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.dropout(x)
        x = x.view(-1, self.fc_input_size)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class CNNAttention(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNAttention, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.attention = nn.Linear(32, 1)
        self.fc = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        attn_weights = self.softmax(self.attention(x.permute(0, 2, 1))).squeeze(-1)
        attn_weights = attn_weights.unsqueeze(-1)
        x = torch.sum(attn_weights * x, dim=2)
        x = self.fc(x)
        return x
    
# Construindo e treinando os modelos

input_size = X.shape[1:]
num_classes = y.shape[1]
hidden_size = 64
num_layers = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convertendo os dados para tensores PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)

# Definindo o tamanho do lote
batch_size = 32

# Criando conjuntos de dados PyTorch
dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Definindo tensor para o LSTM
X_tensorLSTM = X_tensor.permute(0, 2, 1)

# Criando conjuntos de dados PyTorch para LSTM
datasetLSTM = torch.utils.data.TensorDataset(X_tensorLSTM, y_tensor)
train_sizeLSTM = int(0.8 * len(datasetLSTM))
test_sizeLSTM = len(datasetLSTM) - train_sizeLSTM
train_datasetLSTM, test_datasetLSTM = torch.utils.data.random_split(datasetLSTM, [train_sizeLSTM, test_sizeLSTM])

# DataLoader para o LSTM
train_loaderLSTM = DataLoader(train_datasetLSTM, batch_size=batch_size, shuffle=True)
test_loaderLSTM = DataLoader(test_datasetLSTM, batch_size=batch_size, shuffle=False)

# Convertendo os dados para tensores PyTorch do CNN
X_tensorCNN = torch.tensor(X, dtype=torch.float32).to(device)

# Criando conjuntos de dados PyTorch do CNN
datasetCNN = torch.utils.data.TensorDataset(X_tensorCNN, y_tensor)
train_sizeCNN = int(0.8 * len(datasetCNN))
test_sizeCNN = len(datasetCNN) - train_sizeCNN
train_datasetCNN, test_datasetCNN = torch.utils.data.random_split(datasetCNN, [train_sizeCNN, test_sizeCNN])

# DataLoader para o CNN
train_loaderCNN = DataLoader(train_datasetCNN, batch_size=batch_size, shuffle=True)
test_loaderCNN = DataLoader(test_datasetCNN, batch_size=batch_size, shuffle=False)

# Função para treinamento
def train(model, train_loader, criterion, optimizer, num_epochs=100):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            #print(outputs.shape)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

        epoch_loss = running_loss / len(train_loader)
        accuracy = correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}')

In [10]:
# Standard Deep Neural Network
sdnn_model = SimpleDNN(input_size[0], num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(sdnn_model.parameters(), lr=0.001)
train(sdnn_model, train_loader, criterion, optimizer)

Epoch [1/100], Loss: 2.7603, Accuracy: 0.1708
Epoch [2/100], Loss: 1.8016, Accuracy: 0.1730
Epoch [3/100], Loss: 1.7951, Accuracy: 0.1774
Epoch [4/100], Loss: 1.7817, Accuracy: 0.1907
Epoch [5/100], Loss: 1.7577, Accuracy: 0.2201
Epoch [6/100], Loss: 1.7215, Accuracy: 0.2342
Epoch [7/100], Loss: 1.6921, Accuracy: 0.2585
Epoch [8/100], Loss: 1.6724, Accuracy: 0.2676
Epoch [9/100], Loss: 1.6760, Accuracy: 0.2718
Epoch [10/100], Loss: 1.6504, Accuracy: 0.2804
Epoch [11/100], Loss: 1.6290, Accuracy: 0.2972
Epoch [12/100], Loss: 1.6183, Accuracy: 0.3049
Epoch [13/100], Loss: 1.6011, Accuracy: 0.3177
Epoch [14/100], Loss: 1.6020, Accuracy: 0.3158
Epoch [15/100], Loss: 1.5925, Accuracy: 0.3214
Epoch [16/100], Loss: 1.5861, Accuracy: 0.3229
Epoch [17/100], Loss: 1.5869, Accuracy: 0.3259
Epoch [18/100], Loss: 1.5733, Accuracy: 0.3313
Epoch [19/100], Loss: 1.5693, Accuracy: 0.3339
Epoch [20/100], Loss: 1.5734, Accuracy: 0.3429
Epoch [21/100], Loss: 1.5605, Accuracy: 0.3350
Epoch [22/100], Loss: 

In [11]:
# LSTM
lstm_model = LSTMModel(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)
train(lstm_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/100], Loss: 1.6779, Accuracy: 0.2841
Epoch [2/100], Loss: 1.5626, Accuracy: 0.3450
Epoch [3/100], Loss: 1.5328, Accuracy: 0.3620
Epoch [4/100], Loss: 1.5104, Accuracy: 0.3758
Epoch [5/100], Loss: 1.4918, Accuracy: 0.3806
Epoch [6/100], Loss: 1.4637, Accuracy: 0.3990
Epoch [7/100], Loss: 1.4661, Accuracy: 0.4011
Epoch [8/100], Loss: 1.4573, Accuracy: 0.4053
Epoch [9/100], Loss: 1.4430, Accuracy: 0.4038
Epoch [10/100], Loss: 1.4197, Accuracy: 0.4151
Epoch [11/100], Loss: 1.4149, Accuracy: 0.4201
Epoch [12/100], Loss: 1.4139, Accuracy: 0.4238
Epoch [13/100], Loss: 1.4092, Accuracy: 0.4231
Epoch [14/100], Loss: 1.4126, Accuracy: 0.4252
Epoch [15/100], Loss: 1.4041, Accuracy: 0.4285
Epoch [16/100], Loss: 1.3937, Accuracy: 0.4327
Epoch [17/100], Loss: 1.3944, Accuracy: 0.4314
Epoch [18/100], Loss: 1.3718, Accuracy: 0.4366
Epoch [19/100], Loss: 1.3639, Accuracy: 0.4398
Epoch [20/100], Loss: 1.3638, Accuracy: 0.4446
Epoch [21/100], Loss: 1.3636, Accuracy: 0.4462
Epoch [22/100], Loss: 

In [12]:
# LSTM with Attention
lstm_atn_model = LSTMAttention(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_atn_model.parameters(), lr=0.001)
train(lstm_atn_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/100], Loss: 1.6618, Accuracy: 0.2926
Epoch [2/100], Loss: 1.5502, Accuracy: 0.3472
Epoch [3/100], Loss: 1.5184, Accuracy: 0.3722
Epoch [4/100], Loss: 1.4968, Accuracy: 0.3734
Epoch [5/100], Loss: 1.4799, Accuracy: 0.3921
Epoch [6/100], Loss: 1.4602, Accuracy: 0.3929
Epoch [7/100], Loss: 1.4416, Accuracy: 0.4035
Epoch [8/100], Loss: 1.4385, Accuracy: 0.4032
Epoch [9/100], Loss: 1.4308, Accuracy: 0.4065
Epoch [10/100], Loss: 1.4210, Accuracy: 0.4220
Epoch [11/100], Loss: 1.4111, Accuracy: 0.4245
Epoch [12/100], Loss: 1.3947, Accuracy: 0.4277
Epoch [13/100], Loss: 1.4021, Accuracy: 0.4272
Epoch [14/100], Loss: 1.4136, Accuracy: 0.4273
Epoch [15/100], Loss: 1.4554, Accuracy: 0.4092
Epoch [16/100], Loss: 1.4211, Accuracy: 0.4189
Epoch [17/100], Loss: 1.3927, Accuracy: 0.4322
Epoch [18/100], Loss: 1.3865, Accuracy: 0.4344
Epoch [19/100], Loss: 1.3908, Accuracy: 0.4463
Epoch [20/100], Loss: 1.3839, Accuracy: 0.4341
Epoch [21/100], Loss: 1.3655, Accuracy: 0.4546
Epoch [22/100], Loss: 

In [13]:
# Convolutional Neural Network
cnn_model = CNNModel(input_size[0],num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)
train(cnn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/100], Loss: 1.7407, Accuracy: 0.2409
Epoch [2/100], Loss: 1.6430, Accuracy: 0.2960
Epoch [3/100], Loss: 1.6163, Accuracy: 0.3051
Epoch [4/100], Loss: 1.6026, Accuracy: 0.3109
Epoch [5/100], Loss: 1.5752, Accuracy: 0.3239
Epoch [6/100], Loss: 1.5507, Accuracy: 0.3355
Epoch [7/100], Loss: 1.5542, Accuracy: 0.3469
Epoch [8/100], Loss: 1.5676, Accuracy: 0.3286
Epoch [9/100], Loss: 1.5340, Accuracy: 0.3469
Epoch [10/100], Loss: 1.5325, Accuracy: 0.3418
Epoch [11/100], Loss: 1.5237, Accuracy: 0.3518
Epoch [12/100], Loss: 1.5159, Accuracy: 0.3595
Epoch [13/100], Loss: 1.5083, Accuracy: 0.3555
Epoch [14/100], Loss: 1.5055, Accuracy: 0.3576
Epoch [15/100], Loss: 1.5081, Accuracy: 0.3602
Epoch [16/100], Loss: 1.5000, Accuracy: 0.3633
Epoch [17/100], Loss: 1.4946, Accuracy: 0.3741
Epoch [18/100], Loss: 1.4929, Accuracy: 0.3687
Epoch [19/100], Loss: 1.4744, Accuracy: 0.3783
Epoch [20/100], Loss: 1.4751, Accuracy: 0.3843
Epoch [21/100], Loss: 1.4596, Accuracy: 0.3843
Epoch [22/100], Loss: 

In [14]:
# Convolutional Neural Network with Attention
cnn_atn_model = CNNAttention(input_size[0],num_classes).to(device)
optimizer = optim.Adam(cnn_atn_model.parameters(), lr=0.001)
train(cnn_atn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/100], Loss: 1.7092, Accuracy: 0.2671
Epoch [2/100], Loss: 1.5635, Accuracy: 0.3366
Epoch [3/100], Loss: 1.5259, Accuracy: 0.3593
Epoch [4/100], Loss: 1.5164, Accuracy: 0.3659
Epoch [5/100], Loss: 1.4857, Accuracy: 0.3864
Epoch [6/100], Loss: 1.4775, Accuracy: 0.3862
Epoch [7/100], Loss: 1.4642, Accuracy: 0.3943
Epoch [8/100], Loss: 1.4498, Accuracy: 0.4043
Epoch [9/100], Loss: 1.4306, Accuracy: 0.4097
Epoch [10/100], Loss: 1.4298, Accuracy: 0.4075
Epoch [11/100], Loss: 1.4199, Accuracy: 0.4196
Epoch [12/100], Loss: 1.4082, Accuracy: 0.4203
Epoch [13/100], Loss: 1.4085, Accuracy: 0.4237
Epoch [14/100], Loss: 1.4028, Accuracy: 0.4260
Epoch [15/100], Loss: 1.3979, Accuracy: 0.4331
Epoch [16/100], Loss: 1.3849, Accuracy: 0.4319
Epoch [17/100], Loss: 1.3571, Accuracy: 0.4433
Epoch [18/100], Loss: 1.3602, Accuracy: 0.4482
Epoch [19/100], Loss: 1.3606, Accuracy: 0.4458
Epoch [20/100], Loss: 1.3696, Accuracy: 0.4384
Epoch [21/100], Loss: 1.3307, Accuracy: 0.4574
Epoch [22/100], Loss: 

In [15]:
# Função para teste
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

    test_loss = running_loss / len(test_loader)
    test_accuracy = correct / total
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

In [16]:
print("Testing SimpleDNN:")
test(sdnn_model, test_loader, criterion)

Testing SimpleDNN:
Test Loss: 1.4449, Test Accuracy: 0.3962


In [17]:
print("\nTesting LSTMModel:")
test(lstm_model, test_loaderLSTM, criterion)



Testing LSTMModel:
Test Loss: 1.4472, Test Accuracy: 0.4318


In [18]:
print("\nTesting LSTMAttention:")
test(lstm_atn_model, test_loaderLSTM, criterion)


Testing LSTMAttention:
Test Loss: 1.4919, Test Accuracy: 0.4278


In [19]:
print("\nTesting CNNModel:")
test(cnn_model, test_loaderCNN, criterion)



Testing CNNModel:
Test Loss: 1.4901, Test Accuracy: 0.3801


In [20]:
print("\nTesting CNNAttention:")
test(cnn_atn_model, test_loaderCNN, criterion)


Testing CNNAttention:
Test Loss: 1.5998, Test Accuracy: 0.4439
